In [3]:
%cd "C:\Git\BI0730"
import sys
sys.path.append(r'.\.\.\.\.\.')

from libs.geral.utils import *
from libs.geral.df_pipes import *
from libs.geral.myconstants import *
import pandas as pd
import datetime
from icecream import ic
from datetime import datetime, timedelta
import duckdb
import warnings
warnings.filterwarnings("ignore")

print("✅ Librerías cargadas exitosamente")

C:\Git\BI0730
✅ Librerías cargadas exitosamente


In [ ]:
# Configuración de rutas y variables globales
ANO_ACTUAL = 2025
USER = os.getlogin()

# Rutas principales
CAMINHO_PLACAR_GIRO = rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira"
CAMINHO_DADOS_PLACARES = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\_PLACARES\Placar dos Gerentes\dados"
CAMINHO_ARQUIVO_GIRO = os.path.join(PATH_BASES_PARQUET, "giro_de_carteira.parquet")

# Rutas específicas para datos
PASTA_GIRO = os.path.join(CAMINHO_DADOS_PLACARES, "ipp", "giro_de_carteira")
PASTA_ASSOCIADOS = os.path.join(CAMINHO_DADOS_PLACARES, "associados_total")

ic(PASTA_GIRO)
ic(PASTA_ASSOCIADOS)
ic(CAMINHO_ARQUIVO_GIRO)

In [ ]:
# Funciones auxiliares para fechas y mapeo
def final_mes(ano_mes):
    """Obtiene el último día del mes para un año-mes dado"""
    data = ultimo_dia_do_mes(datetime(ano_mes // 100, ano_mes % 100, 1))
    return data

def data_considerada(ano_mes):
    """Determina la fecha a considerar según el año-mes"""
    ano_mes_atual_int = int(ano_mes_atual().replace("-", ""))
    
    if ano_mes != ano_mes_atual_int:
        return final_mes(int(ano_mes))
    else:
        return datetime.today()

# Mapeo de niveles de riesgo
MAP_RISCO = {
    "BAIXÍSSIMO": 0,
    "BAIXO 1": 1,
    "BAIXO 2": 2,
    "MÉDIO 1": 3,
    "MÉDIO 2": 4,
    "ALTO 1": 5,
    "ALTO 2": 6,
    "ALTÍSSIMO": 7,
    "DEFAULT": 8,
}

print("✅ Funciones auxiliares definidas")
ic(MAP_RISCO)

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
# hoje = str(datetime.now().strftime("%Y-%m-%d"))



caminho_placar_gns = rf"C:\Users\{os.getlogin()}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\_PLACARES\Placar dos Gerentes\2024\03_Python_ETL"
caminho_arquivo_giro = rf"C:\Users\{os.getlogin()}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\giro_de_carteira.parquet"
ano = 2025

import sys
sys.path.insert(0,caminho_placar_gns)
from libs import ipps #type: ignore


sys.path.append(rf"C:\Git\BI0730")
from prefect import flow, task, get_run_logger
# print("Iniciando Giro de Carteira")

# Cargar datos de giro de carteira
print("📂 Cargando datos de giro de carteira...")

try:
    # Cargar datos de giro
    giro_raw = carregar_arquivos_da_pasta(PASTA_GIRO)
    ic(f"Registros cargados en giro: {len(giro_raw)}")
    ic(giro_raw.columns.tolist())
    
    # Mostrar muestra de datos
    print("\n📊 Muestra de datos de giro:")
    display(giro_raw.head())
    
    print("\n📈 Información del dataset de giro:")
    print(giro_raw.info())
    
except Exception as e:
    print(f"❌ Error cargando datos de giro: {e}")
    giro_raw = pd.DataFrame()

ipps.carregarDadosGiroDeCarteira(os.path.join(caminho_placar_gns, f'placares/{ano}')).to_parquet(caminho_arquivo_giro)


# init_giro_de_carteira()

            
print("Finalizando Giro de Carteira")

Finalizando Giro de Carteira


In [ ]:

# Cargar datos de associados activos
print("📂 Cargando datos de associados activos...")

try:
    # Cargar asociados
    ativos_raw = carregar_arquivos_da_pasta(PASTA_ASSOCIADOS)
    
    # Filtrar correntistas activos
    ativos = ativos_raw.query("flg_correntista == 'S'")[
        [
            "agencia",
            "carteira", 
            "cpf_cnpj",
            "conta",
            "ano_mes",
            "flg_ativo",
            "dt_ini_asso",
            "des_faixa_risco",
        ]
    ]
    
    # Limpiar y formatear datos
    ativos = (ativos
        .assign(ano_mes = lambda df : df["ano_mes"].astype(int))
        .assign(conta = lambda df : df["conta"].apply(lambda x: str(x).zfill(6)))
    )
    
    ic(f"Registros cargados en ativos: {len(ativos)}")
    ic(ativos.columns.tolist())
    
    print("\n📊 Muestra de datos de associados activos:")
    display(ativos.head())
    
    print(f"\n📈 Información del dataset de associados activos:")
    print(ativos.info())
    
except Exception as e:
    print(f"❌ Error cargando datos de associados: {e}")
    ativos = pd.DataFrame()

ImportError: attempted relative import with no known parent package

In [ ]:
# Procesamiento inicial de datos de giro
print("🔄 Procesando datos de giro...")

if not giro_raw.empty:
    # Ajustar año-mes (mapear 202312 a 202401)
    giro_processed = (giro_raw
        .assign(ano_mes = lambda df: df["ano_mes"].map({"202312": "202401"}).fillna(df["ano_mes"]))
        .assign(ano_mes = lambda df: df["ano_mes"].astype(int))
        .assign(data_ultimo_contato = lambda df: pd.to_datetime(df["data_ultimo_contato"]))
        .groupby(["cpf_cnpj", "ano_mes"], as_index=False)["data_ultimo_contato"].max()
    )
    
    ic(f"Registros después del procesamiento inicial: {len(giro_processed)}")
    print("\n📊 Muestra de datos procesados:")
    display(giro_processed.head())
    
    print(f"\n📅 Rango de fechas en data_ultimo_contato:")
    ic(giro_processed['data_ultimo_contato'].min())
    ic(giro_processed['data_ultimo_contato'].max())
    
else:
    print("❌ No hay datos de giro para procesar")
    giro_processed = pd.DataFrame()

In [ ]:
# Merge de giro con asociados activos
print("🔗 Realizando merge de datos de giro con asociados activos...")

if not giro_processed.empty and not ativos.empty:
    # Merge con asociados activos
    giro_associados = (
        giro_processed.merge(ativos, on=["cpf_cnpj", "ano_mes"], how="right", suffixes=("_giro", ""))
        .assign(dias_associado=lambda df: (datetime.today() - pd.to_datetime(df.dt_ini_asso)).dt.days)
        .assign(flg_ativo=lambda df: df.flg_ativo.map({"S": 1, "N": 0}).fillna(0))
        .set_index("cpf_cnpj")
        .assign(data_ultimo_contato=lambda df: df.groupby(["cpf_cnpj"])["data_ultimo_contato"].ffill())
        .reset_index()
        .assign(data_ultimo_contato=lambda df: pd.to_datetime(df["data_ultimo_contato"]))
        .assign(dt_ini_asso=lambda df: pd.to_datetime(df["dt_ini_asso"]))
    )
    
    ic(f"Registros después del merge: {len(giro_associados)}")
    print("\n📊 Muestra de datos merged:")
    display(giro_associados.head())
    
    print(f"\n📈 Distribución por flg_ativo:")
    ic(giro_associados['flg_ativo'].value_counts())
    
else:
    print("❌ No se puede realizar el merge - faltan datos")
    giro_associados = pd.DataFrame()

In [ ]:
# Cálculos de fechas y giro vencido
print("📅 Calculando fechas y giro vencido...")

if not giro_associados.empty:
    # Combinar fechas y calcular días desde último contacto
    giro_associados['data_ultimo_contato'] = giro_associados['data_ultimo_contato'].combine_first(
        giro_associados['dt_ini_asso']
    ).dt.date
    
    giro_final = (
        giro_associados
        .assign(data_ultimo_contato=lambda df: np.where(
            pd.to_datetime(df["data_ultimo_contato"]) > df["dt_ini_asso"], 
            pd.to_datetime(df["data_ultimo_contato"]).dt.date, 
            df["dt_ini_asso"].dt.date
        ))
        .assign(data_considerada=lambda df: df["ano_mes"].astype(int).apply(data_considerada).dt.date)
        .assign(dias_desde_ult_contato=lambda df: df.apply(
            lambda x: (x["data_considerada"] - x["data_ultimo_contato"]).days 
            if pd.notna(x["data_ultimo_contato"]) else None, axis=1
        ))
        .assign(prazo_giro_em_dias=365)
        .assign(flg_giro_vencido=lambda df: df.apply(
            lambda x: 1 if x["dias_desde_ult_contato"] > x["prazo_giro_em_dias"] else 0, axis=1
        ))
        .assign(dt_ini_asso=lambda df: pd.to_datetime(df['dt_ini_asso']).dt.date)
        .assign(nivel_risco=lambda df: df["des_faixa_risco"].map(MAP_RISCO))
        .query("nivel_risco <= 4")
    )
    
    ic(f"Registros después de cálculos: {len(giro_final)}")
    print("\n📊 Muestra de datos finales:")
    display(giro_final.head())
    
    print(f"\n📈 Distribución de giro vencido:")
    ic(giro_final['flg_giro_vencido'].value_counts())
    
    print(f"\n📈 Distribución por nivel de riesgo:")
    ic(giro_final['nivel_risco'].value_counts())
    
else:
    print("❌ No hay datos para calcular fechas")
    giro_final = pd.DataFrame()

In [ ]:
# Filtrar carteiras válidas y generar resultado final
print("🔍 Aplicando filtros de carteiras válidas...")

if not giro_final.empty:
    # Filtrar carteiras de producción válidas
    giro_carteiras_validas = giro_final[
        ~giro_final['carteira'].str.startswith(('00', 'DIG', 'INV', '99', '6', '5', '4', '188', '10', '350'))
    ]
    
    # Generar resultado agregado por agencia, carteira, ano_mes
    resultado_ipp = (
        giro_carteiras_validas
        .groupby(["agencia", "carteira", "ano_mes"], as_index=False)
        .agg({"flg_ativo": "size", "flg_giro_vencido": "sum"})
        .assign(produto="giro_de_carteira")
        .rename(columns={"flg_ativo": "socios", "flg_giro_vencido": "realizado_soma"})
    )
    
    ic(f"Registros finales en resultado IPP: {len(resultado_ipp)}")
    print("\n📊 Muestra del resultado final:")
    display(resultado_ipp.head(10))
    
    print(f"\n📈 Resumen por agencia:")
    resumen_agencia = resultado_ipp.groupby('agencia').agg({
        'socios': 'sum',
        'realizado_soma': 'sum'
    }).reset_index()
    display(resumen_agencia.head())
    
    # Guardar resultado
    print(f"\n💾 Guardando resultado en: {CAMINHO_ARQUIVO_GIRO}")
    giro_carteiras_validas.to_parquet(CAMINHO_ARQUIVO_GIRO, index=False)
    print("✅ Archivo guardado exitosamente")
    
else:
    print("❌ No hay datos para procesar")
    resultado_ipp = pd.DataFrame()

In [ ]:
# Ejecutar flow completo y validar resultados
print("🚀 Ejecutando flow completo de Giro de Carteira...")

try:
    # Importar el flow desde el archivo de tasks
    from giro_tasks import giro_de_carteira_flow
    
    # Ejecutar el flow
    resultado_flow = giro_de_carteira_flow(ano=2025)
    
    print(f"✅ Flow ejecutado exitosamente")
    print(f"📊 Total de registros procesados: {len(resultado_flow)}")
    
    # Validaciones finales
    if not resultado_flow.empty:
        print("\n📈 Estadísticas finales:")
        print(f"  • Agencias únicas: {resultado_flow['agencia'].nunique()}")
        print(f"  • Carteiras únicas: {resultado_flow['carteira'].nunique()}")
        print(f"  • Meses procesados: {resultado_flow['ano_mes'].nunique()}")
        print(f"  • Total de socios activos: {resultado_flow['flg_ativo'].sum()}")
        print(f"  • Total giros vencidos: {resultado_flow['flg_giro_vencido'].sum()}")
        
        # Mostrar distribución por nivel de riesgo
        print(f"\n📊 Distribución por nivel de riesgo:")
        ic(resultado_flow['nivel_risco'].value_counts().sort_index())
        
        # Verificar que el archivo se guardó
        if os.path.exists(CAMINHO_ARQUIVO_GIRO):
            print(f"\n💾 Archivo guardado exitosamente en: {CAMINHO_ARQUIVO_GIRO}")
            
            # Verificar tamaño del archivo
            file_size = os.path.getsize(CAMINHO_ARQUIVO_GIRO) / (1024 * 1024)  # MB
            print(f"  • Tamaño del archivo: {file_size:.2f} MB")
        else:
            print("❌ El archivo no se guardó correctamente")
    
except Exception as e:
    print(f"❌ Error ejecutando el flow: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
dim = pd.read_parquet(rf'C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\_PLACARES\Placar dos Gerentes\2025\02 Bases\02 Bases Salvas\02_dados_prata\programadas\analitico.parquet')



In [10]:
fact = pd.read_parquet(rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\34_PoupancaProgramada\dados\2023_new\power_bi\eventos_programadas.parquet")

In [14]:
dim.query("conta == '000737'")

,ano_mes,conta,carteira,agencia,cpf_cnpj,nom_associado,sistemas,qtde_mensal,valor_mensal,produto,data_criacao_do_registro
90213,202501,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05
197982,202502,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05
243169,202503,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05
293521,202504,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05
394313,202505,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05
463106,202506,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05
574301,202507,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05
618770,202508,000737,112,14,08038691938,DANIELE FRANCO,poupanca_legado,1,1,programadas,2025-08-07 09:23:05


In [13]:
fact.query("cta_corrente == '000737'")

,cta_corrente,flag_aplicacao,ano_mes,dat_ult_atualizacao,tipo_programada,num_programacao,valor,data_cadastro,num_programacao_legado,data_cadastro_legado,valor_legado,tipo_programada_legado,valor_plataforma,data_cadastro_plataforma,tipo_programada_pf,tipo_movimento,tipo_programada_filtro
67337,000737,1,202508,07-08-2025,nan,nan,nan,NaT,nan,NaT,nan,nan,nan,NaT,nan,N/A,N/A


In [15]:
contas = ler_parquet("contas_analitico")

In [ ]:
cas = cad.query("cod_agencia == '04' and cod_carteira == '611'").rename(columns={'num_cpf_cnpj':'cpf_cnpj'})

In [ ]:
cod_agencia	cpf_cnpj	nom_associado	cod_carteira	nom_carteira

In [32]:
cas.to_excel(rf"C:\Users\carola_luco\OneDrive - Sicredi\Documentos\relatorios\otimiza\contascarteira611_04.xlsx")

In [33]:
carteira_611 =(contas.query("cod_agencia == '04'").merge(cas,on="cpf_cnpj",how='inner'))

In [34]:
carteira_611

,des_marca,cod_cooperativa_x,cod_agencia_x,num_conta_x,cpf_cnpj,tpo_pessoa,dat_abertura,dat_encerramento,des_canal,resp_abertura,des_status_conta_x,des_status_atividade,des_produto,des_modalidade_produto,flg_menor,cod_cooperativa_y,cod_agencia_y,nom_associado,cod_carteira,nom_carteira,nom_gestor_carteira,num_conta_y,flg_tipo_pessoa,des_status_conta_y,dat_inicio_associacao,dat_abertura_conta,dat_encerramento_conta,flg_associado_ativo,flg_conta_corrente,flg_associado_total,flg_conta_capital,flg_conta_salario,flg_conta_poupanca,flg_conta_juridica,flg_fisital,cod_conglomerado_economico,des_conglomerado_economico,des_segmento,des_subsegmento,dat_ultima_atualizacao_conta,data_atualizacao
0,LEGADO,0730,04,266353,08238568951,PF,2008-04-17,None,Agência,None,Ativa,Ativa,Poupança Tradicional,Individual,N,0730,04,LUCAS GOMES SCHMIDT,611,OUTROS,FABIANO NARDI,594893,PF,Conta Cadastrada,None,2021-12-17,None,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,28859,IRAN TAQUES,PESSOA FÍSICA,PF III,2024-09-04,2025-01-11 01:49:22.290
1,LEGADO,0730,04,895738,06234539912,PF,2022-03-25,2023-09-08,Agência,RAUL_GONCALVES,Encerrada,Inativa,Corrente,Conjunta Solidária,N,0730,04,CAMILA PELLIZZA,611,OUTROS,FABIANO NARDI,544464,PF,Conta Cadastrada,2025-01-07,2023-06-23,2023-08-22,NÃO,NÃO,NÃO,SIM,NÃO,NÃO,NÃO,NÃO,31590,CAMILA PELLIZZA SERVICOS MEDICOS LIMITADA,AGRO,GRANDE PRODUTOR,2025-07-25,2025-08-06 12:14:32.120
2,FISITAL,0730,04,555800632,03906835588,PF,2019-03-25,None,Agência,None,Ativa,Ativa,Poupança Tradicional,Individual,N,0730,04,JULIMAR SILVA NASCIMENTO,611,OUTROS,FABIANO NARDI,618272,PF,Conta Cadastrada,None,2024-01-29,None,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,35404,46.459.235 JULIMAR SILVA NASCIMENTO,PESSOA FÍSICA,PF II,2025-02-12,2025-02-13 01:22:56.990
3,LEGADO,0730,04,133596,10072532904,PF,2017-04-11,2021-03-05,Agência,vanessa_pryzbeuka,Encerrada,Inativa,Corrente,Individual,N,0730,04,LUCAS RIBEIRO DA SILVA,611,OUTROS,FABIANO NARDI,495754,PF,Conta Cadastrada,None,2021-11-12,None,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,22046,LUCAS RIBEIRO DA SILVA 10072532904,PESSOA FÍSICA,PF I,2024-09-04,2025-01-11 01:49:22.290
4,FISITAL,0730,04,555236588,09098641954,PF,2020-02-19,None,Agência,None,Ativa,Ativa,Poupança Tradicional,Individual,N,0730,04,MARIANA PASTURCHAK CZEREK,611,OUTROS,FABIANO NARDI,979873,PF,Conta Cadastrada,None,2024-05-29,None,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,37861,CZEREK PAISAGISMO E PLANTAS LTDA,PESSOA FÍSICA,PF II,2024-09-04,2025-01-11 01:49:22.290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,LEGADO,0730,04,560587,78302994987,PF,2024-09-09,None,Agência,LUCAS_CRISTIANO,Ativa,,Capital,Individual,N,0730,04,RONI EMERSON NADAL,611,OUTROS,FABIANO NARDI,560587,PF,Conta Ativa,2024-09-09,2024-09-06,4712-01-01,SIM,SIM,SIM,SIM,NÃO,SIM,NÃO,NÃO,27701,RONI EMERSON NADAL SERVICOS DE MECANICA,PESSOA FÍSICA,PF III,2024-09-11,2025-04-09 12:47:41.260
1060,LEGADO,0730,04,050024,21947805991,PF,2002-02-06,None,Agência,None,Ativa,Inativa,Corrente,Individual,N,0730,04,ADEMAR BACKES,611,OUTROS,FABIANO NARDI,050024,PF,Conta Inativa,None,2002-02-06,2003-09-18,NÃO,SIM,NÃO,SIM,NÃO,NÃO,NÃO,NÃO,None,,PESSOA FÍSICA,PF I,2024-09-04,2025-01-11 01:49:22.290
1061,LEGADO,0730,04,053546,34843566934,PF,2005-02-01,None,Agência,None,Ativa,Inativa,Corrente,Individual,N,0730,04,JOSE CARLOS HOLK,611,OUTROS,FABIANO NARDI,053546,PF,Conta Inativa,None,2005-02-01,2007-04-03,NÃO,SIM,NÃO,SIM,NÃO,NÃO,NÃO,NÃO,,,PESSOA FÍSICA,PF III,2024-09-04,2025-01-11 01:49:22.290
1062,LEGADO,0730,04,495381,70570493072,PF,2024-01-10,None,Agência,None,Ativa,Ativa,Poupança Tradicional,Individual,N,0730,04,MARCOS ROBERTO DE ALCANTARA,611,OUTROS,FABIANO NARDI,053201,PF,Conta Inativa,None,2005-01-18,2007-03-30,NÃO,SIM,NÃO,SIM,NÃO,NÃO,NÃO,NÃO,None,,PESSOA FÍSICA,PF II,2024-09-04,2025-01-11 01:49:22.290
